# **Are company providing sufficient mental health support?**


![Mental Health](src/pic/pic.png)


Mental health is essential because it affects every aspect of our lives, from how we think and feel to how we interact with others and handle challenges. Good mental health enables us to cope with stress, build healthy relationships, and make meaningful contributions to our communities. Prioritizing mental health promotes overall well-being, boosts productivity, and enhances quality of life. Ignoring it can lead to serious emotional, physical, and social consequences. Taking care of our mental health is not just self-care; it’s a foundation for living a balanced and fulfilling life.


---


In [1]:
import sqlite3
import duckdb
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go

In [2]:
from src.func.chart_functions import bar_chart

# **Connect to DB**


In [3]:
conn = sqlite3.connect("src/db/mental_health.sqlite")
df = pd.read_sql(
    """
                SELECT 
                    a.UserID
                    , a.SurveyID AS 'Year'
                    , a.QuestionID 
                    , q.questiontext 
                    , a.AnswerText 

                FROM Answer AS a 
                    LEFT JOIN Question AS q 
                    ON a.questionid = q.questionid;""",
    conn,
)

# **EDA**


## **Understanding the Respondents**


### **How many users was in each year**


In [4]:
users_df = duckdb.sql(
    """
    SELECT 
        DISTINCT Year, 
        count(DISTINCT UserID) AS UserID_count
    FROM df
    GROUP BY Year
"""
).to_df()

In [5]:
bar_chart(df=users_df, xaxis="Year", yaxis="UserID_count", title="Unique Users by Year")

### **Users Sociodemographics**


In [6]:
# Create table with age, gender and living location
demografic_df = duckdb.sql(
    """
WITH
    age AS (
        SELECT UserID, Year, CAST(AnswerText AS INT64) AS Age
        FROM df
        WHERE QuestionID = 1
        ),
    gender AS (
        SELECT UserID, Year, LOWER(AnswerText) AS Gender
        FROM df
        WHERE QuestionID = 2
        ),
    location AS (
        SELECT UserID, Year, AnswerText AS Location
        FROM df
        WHERE QuestionID = 3
        ),
    final AS (
        SELECT age.UserID, age.Year, age.Age, gender.Gender, location.Location
        FROM age
            LEFT JOIN gender
            ON age.UserID = gender.UserID

            LEFT JOIN location
            ON age.UserID = location.UserID 
    )

SELECT * FROM final;
"""
).to_df()

In [7]:
demografic_df = duckdb.sql(
    """
    SELECT *

    FROM demografic_df 

    WHERE
        Age BETWEEN 18 AND 67 
        AND Gender IN ('male', 'female')
    """
).to_df()

In [8]:
# Explore Age column
px.histogram(data_frame=demografic_df, x="Age", color="Gender")

Based on the distribution, we can see that the largest portion falls within the 25-40 age range.


In [9]:
# Explore Location column
demografic_df["Location"] = demografic_df["Location"].replace(
    "United States", "United States of America"
)
demografic_df = demografic_df.loc[demografic_df["Location"] != "Other"]

In [10]:
location_map = {
    "United States of America": "North America",
    "United Kingdom": "Europe",
    "Canada": "North America",
    "Germany": "Europe",
    "Netherlands": "Europe",
    "Australia": "Australia/Oceania",
    "France": "Europe",
    "Ireland": "Europe",
    "India": "Asia",
    "Brazil": "South America",
    "Sweden": "Europe",
    "Spain": "Europe",
    "Switzerland": "Europe",
    "New Zealand": "Australia/Oceania",
    "Poland": "Europe",
    "Portugal": "Europe",
    "Italy": "Europe",
    "Belgium": "Europe",
    "South Africa": "Africa",
    "Russia": "Asia",
    "Bulgaria": "Europe",
    "Norway": "Europe",
    "Mexico": "North America",
    "Finland": "Europe",
    "Israel": "Asia",
    "Denmark": "Europe",
    "Japan": "Asia",
    "Romania": "Europe",
    "Austria": "Europe",
    "Greece": "Europe",
    "Pakistan": "Asia",
    "Colombia": "South America",
    "Czech Republic": "Europe",
    "Estonia": "Europe",
    "Turkey": "Asia",
    "Singapore": "Asia",
    "Hungary": "Europe",
    "Argentina": "South America",
    "Croatia": "Europe",
    "Serbia": "Europe",
    "Ukraine": "Europe",
    "Bangladesh": "Asia",
    "Chile": "South America",
    "Bosnia and Herzegovina": "Europe",
    "Iceland": "Europe",
    "Lithuania": "Europe",
    "Costa Rica": "North America",
    "Afghanistan": "Asia",
    "Algeria": "Africa",
    "Nigeria": "Africa",
    "Indonesia": "Asia",
    "China": "Asia",
    "Georgia": "Asia",
    "Hong Kong": "Asia",
    "Uruguay": "South America",
    "Slovakia": "Europe",
    "Brunei": "Asia",
    "Iran": "Asia",
    "Vietnam": "Asia",
    "Ecuador": "South America",
    "Venezuela": "South America",
    "Guatemala": "North America",
    "Philippines": "Asia",
    "Belarus": "Europe",
    "Moldova": "Europe",
    "Thailand": "Asia",
    "Latvia": "Europe",
    "Slovenia": "Europe",
    "Ghana": "Africa",
    "Macedonia": "Europe",
    "Saudi Arabia": "Asia",
    "Jordan": "Asia",
    "Ethiopia": "Africa",
    "Kenya": "Africa",
    "Mauritius": "Africa",
    "Taiwan": "Asia",
}

In [11]:
demografic_df["Continent"] = demografic_df["Location"].map(location_map)
continent_df = demografic_df["Continent"].value_counts().reset_index()

In [12]:
bar_chart(
    df=continent_df, xaxis="Continent", yaxis="count", title="Distribution by Continent"
)

Based on the available data, we will use only North America and Europe data for comparison.\
There is insufficient data from other continents to conduct a high-quality analysis.


In [13]:
demografic_df = duckdb.sql(
    """
    SELECT * 

    FROM demografic_df 

    WHERE 
        Continent IN ('North America', 'Europe')
    
    ORDER BY Gender ASC
"""
).to_df()

In [14]:
# Explore Gender column
gender_df = demografic_df["Gender"].value_counts().reset_index()

In [15]:
bar_chart(df=gender_df, xaxis="Gender", yaxis="count", title="Distribution by Gender")

We can see that more men work in the IT sector compared to women. According to statistical data, this result is favorable, as women typically make up 20-30% of the IT workforce. Therefore, we can dismiss any significant bias in this context.


In [16]:
# Male and Female qty and procentage
gender_gr = duckdb.sql(
    """
    WITH 
        male_df AS (
            SELECT
                Year, 
                COUNT(Gender) AS 'Male'

            FROM demografic_df 

            WHERE Gender = 'male'

            GROUP BY Year
        ), 

        female_df AS (
            SELECT 
                Year,
                COUNT(Gender) AS 'Female'
            FROM demografic_df 
            WHERE Gender = 'female' 
            GROUP BY Year
        ),

        app_df AS (
            SELECT 
                m.Year, 
                m.Male,
                f.Female

            FROM male_df AS m 
                LEFT JOIN female_df AS f 
                ON m.Year = f.Year
        ), 

        final_df AS (
            SELECT
                Year, 
                Male,
                Female, 
                ROUND(Male / (Male + Female) * 100, 2) AS 'Male_proc',
                ROUND(Female / (Male + Female) * 100, 2) AS 'Female_proc'
            
            FROM app_df 
        )
     
SELECT * FROM final_df 
ORDER BY Year ASC
"""
).to_df()

In [17]:
fig = go.Figure()
fig.add_trace(
    go.Bar(
        x=gender_gr["Year"],
        y=gender_gr["Male"],
        name="Male",
        marker_color="indianred",
        text=gender_gr["Male"],
    )
)

fig.add_trace(
    go.Bar(
        x=gender_gr["Year"],
        y=gender_gr["Female"],
        name="Female",
        marker_color="lightsalmon",
        text=gender_gr["Female"],
    )
)

fig.update_layout(
    title=dict(text="Male And Female Distribution by Year"),
    xaxis=dict(title=dict(text="Year")),
)

In [18]:
fig = go.Figure()
fig.add_trace(
    go.Scatter(
        x=gender_gr["Year"],
        y=gender_gr["Male_proc"],
        name="Male",
        marker_color="indianred",
        text=gender_gr["Male_proc"],
        textposition="top center",
        mode="text+lines+markers",
    )
)

fig.add_trace(
    go.Scatter(
        x=gender_gr["Year"],
        y=gender_gr["Female_proc"],
        name="Female",
        marker_color="lightsalmon",
        text=gender_gr["Female_proc"],
        textposition="top center",
        mode="text+lines+markers",
    )
)

fig.update_layout(
    title="Percentage Distribution of Male and Female",
    xaxis=dict(title=dict(text="Year")),
)

Looking year by year, we see that the ratio of men to women remains at a similar level.


The target group consists of:

- Men and women
- Living in Europe and North America
- Aged between 18 and 67 years


## **Prevalence of mental health conditions**


### **How many respondents currently have or have had mental health issues?**


In [19]:
have_disorder_df = duckdb.sql(
    """
        SELECT
            d.UserID,
            d.Year,
            d.Age,
            d.Gender,
            d.Location,
            d.Continent,
            CASE 
                WHEN hd.Have_disorder = 'Possibly' THEN 'Don''t Know'
                WHEN hd.Have_disorder = 'Maybe' THEN 'Don''t Know' 
                ELSE hd.Have_disorder
                END AS Have_disorder

        FROM demografic_df AS d 
            LEFT JOIN (
                SELECT 
                    UserID, 
                    AnswerText AS 'Have_disorder' 
                FROM df 
                WHERE QuestionID = 33
                ) AS hd 
            ON d.UserID = hd.UserID 

        WHERE
            d.Year != 2014
    """
).to_df()

In [20]:
have_disorder_gr = duckdb.sql(
    """
    WITH 
        main_df AS (
            SELECT 
                Have_disorder, 
                COUNT(Have_disorder) AS 'count',
                (SELECT COUNT(Have_disorder) FROM have_disorder_df) AS 'Total'

            FROM have_disorder_df

            GROUP BY Have_disorder
        ),

        final_df AS (
            SELECT
                Have_disorder,
                count,
                Total,
                ROUND(count / Total * 100, 2) AS 'count%'

            FROM main_df
        )

SELECT * FROM final_df
ORDER BY count DESC
"""
).to_df()

In [21]:
bar_chart(
    df=have_disorder_gr,
    xaxis="Have_disorder",
    yaxis="count%",
    title="Percentage Distribution of Do you currently have a mental health disorder?",
)

A significant portion of respondents reported these disorders. We will later analyze this data across various dimensions to identify the most vulnerable groups.


In [22]:
male_disorder_df = duckdb.sql(
    """
    WITH 
        main_df AS (
            SELECT 
                Have_disorder,
                count(Gender) AS count,
                (SELECT COUNT(Gender) FROM have_disorder_df WHERE Gender = 'male') AS 'Total'

            FROM have_disorder_df

            WHERE
                Gender = 'male'

            GROUP BY Have_disorder
        ),

        final_df AS (
            SELECT
                Have_disorder,
                count,
                Total, 
                ROUND(count / Total * 100, 2) AS 'count%'

            FROM main_df
        )

SELECT * FROM final_df
ORDER BY count DESC
    """
).to_df()

In [23]:
female_disorder_df = duckdb.sql(
    """
    WITH 
        main_df AS (
            SELECT 
                Have_disorder,
                count(Gender) AS 'count',
                (SELECT COUNT(Gender) FROM have_disorder_df WHERE Gender = 'female') AS 'Total'

            FROM have_disorder_df

            WHERE
                Gender = 'female'

            GROUP BY Have_disorder
        ),

        final_df AS (
            SELECT
                Have_disorder,
                count,
                Total,
                ROUND(count / Total * 100, 2) AS 'count%' 

            FROM main_df
        )

SELECT * FROM final_df  
ORDER BY count DESC
    """
).to_df()

In [24]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=male_disorder_df["Have_disorder"],
        y=male_disorder_df["count%"],
        name="Male",
        marker_color="indianred",
        text=male_disorder_df["count%"],
    )
)

fig.add_trace(
    go.Bar(
        x=female_disorder_df["Have_disorder"],
        y=female_disorder_df["count%"],
        name="Female",
        marker_color="lightsalmon",
        text=female_disorder_df["count%"],
    )
)

fig.update_layout(
    title="Percentage Distribution of Male and Female",
    xaxis=dict(title=dict(text="Have Disorder")),
)

The women's group has the highest number of reported disorders, even though they are fewer in number. This could be attributed to experiencing more stressful situations at work.


In [25]:
px.histogram(have_disorder_df, x="Age", color="Have_disorder", title="Disorder by Age")

The largest group experiencing disorders is the most productive age group, between 25 and 40 years old.\
Given this situation, companies should focus more on supporting their employees to ensure well-being and productivity.


In [43]:
# Disorder between continents
continent_disorder_df = duckdb.sql(
    """
    WITH 
        na_main_df AS (
            SELECT
                Have_disorder,
                COUNT(Continent) AS 'North_America',
                (SELECT COUNT(Continent) FROM have_disorder_df WHERE Continent = 'North America') AS 'Total' 
                
            FROM have_disorder_df

            WHERE 
                Continent = 'North America'

            GROUP BY Have_disorder
        ), 

        na_final_df AS (
            SELECT
                Have_disorder,
                ROUND(North_America / Total * 100, 2) AS 'North_America_proc'

            FROM na_main_df 
        ),

        eu_main_df AS (
            SELECT
                Have_disorder,
                COUNT(Continent) AS 'Europe', 
                (SELECT COUNT(Continent) FROM have_disorder_df WHERE Continent = 'Europe') AS 'Total'

            FROM have_disorder_df 

            WHERE Continent = 'Europe' 
            GROUP BY Have_disorder
        ),

        eu_final_df AS (
            SELECT 
                Have_disorder,
                ROUND(Europe / Total * 100, 2) AS 'Europe_proc' 

            FROM eu_main_df 
        ),

        final_df AS (
            SELECT
                na.Have_disorder,
                na.North_America_proc,
                eu.Europe_proc,
                CASE 
                    WHEN na.Have_disorder = 'Yes' THEN 1 
                    WHEN na.Have_disorder = 'No' THEN 2
                    Else 3 
                    END AS 'order_col'

            FROM na_final_df AS na 
                LEFT JOIN eu_final_df AS eu 
                ON na.Have_disorder = eu.Have_disorder
        )
    
    SELECT * FROM final_df 
    ORDER BY order_col ASC
"""
).to_df()

In [44]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=continent_disorder_df["Have_disorder"],
        y=continent_disorder_df["North_America_proc"],
        name="North America",
        marker_color="indianred",
        text=continent_disorder_df["North_America_proc"],
    )
)

fig.add_trace(
    go.Bar(
        x=continent_disorder_df["Have_disorder"],
        y=continent_disorder_df["Europe_proc"],
        name="Europe",
        marker_color="lightsalmon",
        text=continent_disorder_df["Europe_proc"],
    )
)

fig.update_layout(
    title="Percentage Distribution by Continent",
    xaxis=dict(title=dict(text="Have Disorder")),
)

Comparing North America with Europe, we see a higher number of cases in North America. This could be influenced by factors such as work culture, competitiveness, and similar reasons.


## **Workplace perceptions**


### **Are workplaces providing sufficient mental health support?**


In [28]:
workplaces_df = duckdb.sql(
    """
        SELECT 
            h.UserID,
            h.Year,
            h.Age,
            h.Gender,
            h.Location,
            h.Continent,
            h.Have_disorder,
            CASE 
                WHEN c.Health_benefits = 'Not eligible for coverage / NA' THEN 'No'
                ELSE c.Health_benefits
                END AS Health_benefits,

            mhs.Mental_health_services,
            fmh.Formal_mental_health,
            hr.Health_resources

        FROM have_disorder_df AS h 
            -- Does your employer provide mental health benefits as part of healthcare coverage?
            LEFT JOIN (
                SELECT 
                    UserID, 
                    AnswerText AS Health_benefits
                FROM df 
                WHERE QuestionID = 10
            ) AS c 
            ON h.UserID = c.UserID 

            -- Do you know the options for mental health care available under your employer-provided health coverage?
            LEFT JOIN (
                SELECT 
                    UserID,
                    AnswerText AS Mental_health_services
                FROM df 
                WHERE QuestionID = 14
            ) AS mhs 
            ON h.UserID = mhs.UserID 

            -- Has your employer ever formally discussed mental health (for example, as part of a wellness campaign or other official communication)?
            LEFT JOIN (
                SELECT 
                    UserID, 
                    AnswerText AS Formal_mental_health
                FROM df 
                WHERE QuestionID = 15            
            ) AS fmh
            ON h.UserID = fmh.UserID

            -- Does your employer offer resources to learn more about mental health disorders and options for seeking help?
            LEFT JOIN (
                SELECT 
                    UserID,
                    AnswerText AS Health_resources 
                FROM df
                WHERE QuestionID = 16
            ) AS hr 
            ON h.UserID = hr.UserID 

        WHERE
            c.Health_benefits != '-1'
            AND mhs.Mental_health_services != '-1'
            AND fmh.Formal_mental_health != '-1'
            AND hr.Health_resources != '-1'
    """
).to_df()

#### **Health benefits**


In [29]:
# Total distribution of health benefits
health_summary = duckdb.sql(
    """
WITH 
    group_data AS (
        SELECT
            Health_benefits,
            COUNT(Health_benefits) AS count
        
        FROM workplaces_df

        GROUP BY Health_benefits
        ),
    sum_data AS (
        SELECT 
            COUNT(Health_benefits) AS Total
        FROM workplaces_df     
        ),
    app_tbl AS (
            SELECT
                gd.Health_benefits,
                gd.count,
                (SELECT * FROM sum_data) AS Total

            FROM group_data AS gd
        ),
    prc_calc AS (
            SELECT 
                Health_benefits,
                ROUND(count / Total * 100, 2) AS 'count%',
                CASE 
                    WHEN Health_benefits = 'Yes' THEN 1 
                    WHEN Health_benefits = 'No' THEN 2 
                    ELSE 3 
                    END AS 'order_col' 

            FROM app_tbl
        )

SELECT * FROM prc_calc 
ORDER BY order_col ASC
"""
).to_df()

In [30]:
bar_chart(
    df=health_summary,
    xaxis="Health_benefits",
    yaxis="count%",
    title="Total Overwiev of Health Benefits",
)

We see that a large portion includes mental health care services as part of health insurance. However, it is concerning that nearly 30% of respondents are unaware or have not inquired about this service.


In [64]:
# Health benefits distribution by gender
health_benefits_df = duckdb.sql(
    """
WITH 
    male_df AS (
        SELECT
            Health_benefits,
            COUNT(Gender) AS 'Male',
            (SELECT COUNT(Gender) FROM workplaces_df WHERE Gender = 'male') AS 'Total'
        FROM workplaces_df 
        WHERE Gender = 'male'
        GROUP BY Health_benefits
        ),

    male_final_df AS (
        SELECT 
            Health_benefits, 
            ROUND(Male / Total * 100, 2) AS 'Male_proc'
        FROM male_df
        ),
    female_df AS (
        SELECT
            Health_benefits,
            COUNT(Gender) AS 'Female',
            (SELECT COUNT(Gender) FROM workplaces_df WHERE Gender = 'female') AS 'Total'
        FROM workplaces_df 
        WHERE Gender = 'female'
        GROUP BY Health_benefits
    ),
    female_final_df AS (
        SELECT 
            Health_benefits, 
            ROUND(Female / Total * 100, 2) AS 'Female_proc'
        FROM female_df
    ),
    final_df AS (
        SELECT 
            m.Health_benefits,
            m.Male_proc AS 'Male%',
            f.Female_proc AS 'Female%', 
            CASE 
                WHEN m.Health_benefits = 'Yes' THEN 1 
                WHEN m.Health_benefits = 'No' THEN 2
                ELSE 3 
                END AS 'order_col' 

        FROM male_final_df AS m
            LEFT JOIN female_final_df AS f 
            ON m.Health_benefits = f.Health_benefits
    )

SELECT * FROM final_df 
ORDER BY order_col ASC
"""
).to_df()

In [66]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=health_benefits_df["Health_benefits"],
        y=health_benefits_df["Male%"],
        name="Male",
        text=health_benefits_df["Male%"],
        marker_color="indianred",
    )
)

fig.add_trace(
    go.Bar(
        x=health_benefits_df["Health_benefits"],
        y=health_benefits_df["Female%"],
        name="Female",
        text=health_benefits_df["Female%"],
        marker_color="lightsalmon",
    )
)

fig.update_layout(title="Health Benefits Distribution by Gender")

We observe that men tend to care less about their psychological health.


In [67]:
# Health benefits distribution by continent
benefits_continent_df = duckdb.sql(
    """
    WITH 
        /* North America part */
        na_main_df AS (
            SELECT 
                Health_benefits,
                COUNT(Continent) AS 'North_America',
                (SELECT COUNT(Continent) FROM workplaces_df WHERE Continent = 'North America') AS 'Total'

            FROM workplaces_df

            WHERE Continent = 'North America'

            GROUP BY Health_benefits
        ), 

        na_final_df AS (
            SELECT
                Health_benefits, 
                ROUND(North_America / Total * 100, 2) AS 'North_America_proc' 

            FROM na_main_df 
        ),
        /* Europe part */

        eu_main_df AS (
            SELECT 
                Health_benefits,
                COUNT(Continent) AS 'Europe',
                (SELECT COUNT(Continent) FROM workplaces_df WHERE Continent = 'Europe') AS 'Total'

            FROM workplaces_df

            WHERE Continent = 'Europe'

            GROUP BY Health_benefits 
        ), 
        eu_final_df AS (
            SELECT 
                Health_benefits,
                ROUND(Europe / Total * 100, 2) AS 'Europe_proc' 

            FROM eu_main_df
        ),
        /* Final df */
        final_df AS (
            SELECT 
                na.Health_benefits,
                na.North_America_proc,
                eu.Europe_proc,
                CASE 
                    WHEN na.Health_benefits = 'Yes' THEN 1 
                    WHEN na.Health_benefits = 'No' THEN 2 
                    ELSE 3 
                    END AS 'order_col' 

            FROM na_final_df AS na 
                LEFT JOIN eu_final_df AS eu 
                ON na.Health_benefits = eu.Health_benefits
        )


    SELECT * FROM final_df 
    ORDER BY order_col ASC
"""
).to_df()

In [68]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=benefits_continent_df["Health_benefits"],
        y=benefits_continent_df["North_America_proc"],
        name="North America",
        text=benefits_continent_df["North_America_proc"],
        marker_color="indianred",
    )
)

fig.add_trace(
    go.Bar(
        x=benefits_continent_df["Health_benefits"],
        y=benefits_continent_df["Europe_proc"],
        name="Europe",
        text=benefits_continent_df["Europe_proc"],
        marker_color="lightsalmon",
    )
)

fig.update_layout(title="Benefit Distribution by Continent")

North America places more emphasis on health compared to Europe.


### **Do you know the options for mental health care available under your employer-provided health coverage?**


In [35]:
overwiev_mental_helth_service = duckdb.sql(
    """
    WITH 
        main_df AS (
            SELECT
                Mental_health_services,
                COUNT(Mental_health_services) AS 'count'
            FROM workplaces_df

            GROUP BY Mental_health_services
        ), 

        final_df AS (
            SELECT *
            , CASE 
                WHEN Mental_health_services = 'Yes' THEN 1 
                WHEN Mental_health_services = 'No' THEN 2 
                ELSE 3 
                END AS 'order_col' 
            
            FROM main_df
        )
    SELECT * FROM final_df 
    ORDER BY order_col ASC
"""
).to_df()

In [36]:
bar_chart(
    df=overwiev_mental_helth_service,
    xaxis="Mental_health_services",
    yaxis="count",
    title="Overwiev of options for mental health care available under your employer-provided health coverage",
)

We observe that respondents are unaware of the mental health care services provided by their employer.


#### **Mental health services by Gender**


In [85]:
gender_services_df = duckdb.sql(
    """
    WITH
        /* Male part */ 
        male_main_df AS (
            SELECT
                Mental_health_services,
                COUNT(Gender) AS 'Male',
                (SELECT COUNT(Gender) FROM workplaces_df WHERE Gender = 'male') AS 'Total'
            
            FROM workplaces_df 

            WHERE 
                Gender = 'male' 
            
            GROUP BY Mental_health_services
        ),
        male_final_df AS (
            SELECT 
                Mental_health_services,
                ROUND(Male / Total * 100, 2) AS 'Male_proc'

            FROM male_main_df
        ), 

        /* Female part */
        female_main_df AS (
            SELECT
                Mental_health_services,
                COUNT(Gender) AS 'Female',
                (SELECT COUNT(Gender) FROM workplaces_df WHERE Gender = 'female') AS 'Total'
            
            FROM workplaces_df 

            WHERE 
                Gender = 'female' 
            
            GROUP BY Mental_health_services 
        ), 

        female_final_df AS (
            SELECT 
                Mental_health_services, 
                ROUND(Female / Total * 100, 2) AS 'Female_proc' 

            FROM female_main_df
        ),
        /* Merge male and female tables */
        final_df AS (
            SELECT 
                m.Mental_health_services,
                m.Male_proc,
                f.Female_proc,
                CASE 
                    WHEN m.Mental_health_services = 'Yes' THEN 1 
                    WHEN m.Mental_health_services = 'No' THEN 2 
                    ELSE 3 
                    END AS 'order_col' 

            FROM male_final_df AS m 
                LEFT JOIN female_final_df AS f 
                ON m.Mental_health_services = f.Mental_health_services
        )

    SELECT * FROM final_df
    ORDER BY order_col ASC
"""
).to_df()

In [87]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=gender_services_df["Mental_health_services"],
        y=gender_services_df["Male_proc"],
        name="Male",
        text=gender_services_df["Male_proc"],
        marker_color="indianred",
    )
)

fig.add_trace(
    go.Bar(
        x=gender_services_df["Mental_health_services"],
        y=gender_services_df["Female_proc"],
        name="Female",
        text=gender_services_df["Female_proc"],
        marker_color="lightsalmon",
    )
)

fig.update_layout(title="Mental Health Services Distribution by Gender")

Yes, we notice that men are less concerned about and less interested in the mental health care options provided under their employer-sponsored health insurance.


In [88]:
# Services by Continent
continent_service_df = duckdb.sql(
    """
    WITH 
        /* North America part */
        na_service_df AS (
            SELECT 
                Mental_health_services,
                COUNT(Continent) AS 'North_America',
                (SELECT COUNT(Continent) FROM workplaces_df WHERE Continent = 'North America') AS 'Total'

            FROM workplaces_df
            WHERE Continent = 'North America'
            GROUP BY Mental_health_services
        ),
        na_final_df AS (
            SELECT
                Mental_health_services, 
                ROUND(North_America / Total * 100, 2) AS 'North_America_proc'

            FROM na_service_df
        ), 

        /* Europe part */
        eu_service_df AS (
            SELECT 
                Mental_health_services,
                COUNT(Continent) AS 'Europe',
                (SELECT COUNT(Continent) FROM workplaces_df WHERE Continent = 'Europe') AS 'Total'

            FROM workplaces_df
            WHERE Continent = 'Europe'
            GROUP BY Mental_health_services 
        ),
        eu_final_df AS (
            SELECT
                Mental_health_services, 
                ROUND(Europe / Total * 100, 2) AS 'Europe_proc'

            FROM eu_service_df 
        ),
        /* Final df */
        final_df AS (
            SELECT 
                na.Mental_health_services,
                na.North_America_proc,
                eu.Europe_proc,
                CASE 
                    WHEN na.Mental_health_services = 'Yes' THEN 1
                    WHEN na.Mental_health_services = 'No' THEN 2
                    ELSE 3 
                    END AS 'order_col' 

            FROM na_final_df AS na 
                LEFT JOIN eu_final_df AS eu 
                ON na.Mental_health_services = eu.Mental_health_services
        )
       
    SELECT * FROM final_df
    ORDER BY order_col ASC
"""
).to_df()

In [89]:
fig = go.Figure()

fig.add_trace(
    go.Bar(
        x=continent_service_df["Mental_health_services"],
        y=continent_service_df["North_America_proc"],
        name="North America",
        text=continent_service_df["North_America_proc"],
        marker_color="indianred",
    )
)

fig.add_trace(
    go.Bar(
        x=continent_service_df["Mental_health_services"],
        y=continent_service_df["Europe_proc"],
        name="Europe",
        text=continent_service_df["Europe_proc"],
        marker_color="lightsalmon",
    )
)

fig.update_layout(title="Mental Health Services Distribution by Continent")

North America have more options for mental health care available under your employer-provided health coverage


We observe that, when comparing continents, there is greater concern for mental health in North America than in Europe. It is also notable that women are more aware than men of the insurance services offered by their employer.
